<a href="https://colab.research.google.com/github/MathewBiddle/sandbox/blob/main/notebooks/CKAN_API_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Search [IOOS Data Catalog](https://data.ioos.us/) for CF standard names that contain `oxygen` and `sea_water`.

In [1]:
try:
   import ckanapi
except:
   !pip install ckanapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ckanapi: filename=ckanapi-4.7-py3-none-any.whl size=43312 sha256=457bd3c63ca282d57c37e583779d8ad2587acf3e751577ecc5f5f7228fb9181a
  Stored in directory: /root/.cache/pip/wheels/0d/b2/c7/219cd5a752c2ff4fb9809216307d26f6421f6711e0f4e010ff
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=84636d1efd3afc0823add5e223c29697a1dafdff9b4f57740bfdc5bca82b4947
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built ckanapi docopt


Build CKAN API query.

In [2]:
from ckanapi import RemoteCKAN
ua = 'ckanapiioos/1.0 (+https://ioos.us/)'

#ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua, get_only=True)
ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua)
ioos_catalog

Tell me what organizations are there.

In [3]:
orgs = ioos_catalog.action.organization_list()
print(orgs)

['aoos', 'caricoos', 'cdip', 'cencoos', 'comt', 'gcoos', 'glider-dac', 'glos', 'hf-radar-dac', 'ioos', 'maracoos', 'nanoos', 'neracoos', 'noaa-co-ops', 'noaa-ndbc', 'oceansites', 'pacioos', 'sccoos', 'secoora', 'unidata', 'usgs', 'us-navy']


Grab all the datasets available.

In [4]:
#datasets = ioos_catalog.action.package_search(fq='+cf_standard_names:mass_concentration_of_oxygen_in_sea_water', rows=50)
datasets = ioos_catalog.action.package_search()
datasets['count']

10177

Collect CF standard names that contain `oxygen` and `sea_water` from the CF standard name list.

In [5]:
import pandas as pd

url = "https://cfconventions.org/Data/cf-standard-names/current/src/cf-standard-name-table.xml"

df = pd.read_xml(url, xpath="entry")

std_names = df.loc[(df['id'].str.contains('oxygen') & df['id'].str.contains('sea_water'))]

std_names

,id,canonical_units,grib,amip,description
444,depth_at_shallowest_local_minimum_in_vertical_...,m,None,None,Depth is the vertical distance below the surfa...
588,fractional_saturation_of_oxygen_in_sea_water,1,None,None,Fractional saturation is the ratio of some mea...
1274,mass_concentration_of_oxygen_in_sea_water,kg m-3,None,None,Mass concentration means mass per unit volume ...
1616,mole_concentration_of_dissolved_molecular_oxyg...,mol m-3,None,None,Mole concentration means number of moles per u...
1617,mole_concentration_of_dissolved_molecular_oxyg...,mol m-3,None,None,"""Mole concentration at saturation"" means the m..."
1618,mole_concentration_of_dissolved_molecular_oxyg...,mol m-3,None,None,Mole concentration means number of moles per u...
1848,moles_of_oxygen_per_unit_mass_in_sea_water,mol kg-1,None,None,"moles_of_X_per_unit_mass_inY is also called ""m..."
2975,surface_molecular_oxygen_partial_pressure_diff...,Pa,None,None,"The surface called ""surface"" means the lower b..."
3460,temperature_of_sensor_for_oxygen_in_sea_water,K,None,None,Temperature_of_sensor_for_oxygen_in_sea_water ...
4460,volume_fraction_of_oxygen_in_sea_water,1,None,None,"""Volume fraction"" is used in the construction ..."


Search IOOS Data Catalog for CF standard names that match those above.

In [6]:
from ckanapi import RemoteCKAN
ua = 'ckanapiioos/1.0 (+https://ioos.us/)'

#ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua, get_only=True)
ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua)
ioos_catalog

for std_name in std_names['id']:

  print(std_name)

  fq = '+cf_standard_names:{}'.format(std_name)

  datasets = ioos_catalog.action.package_search(fq=fq)

  print('found {} dataset(s)'.format(datasets['count']))

  if datasets['count'] > 0:
    for dataset in datasets['results']:
      print('\t',dataset['title'],dataset['resources'][0]['url'])
  

depth_at_shallowest_local_minimum_in_vertical_profile_of_mole_concentration_of_dissolved_molecular_oxygen_in_sea_water
found 0 dataset(s)
fractional_saturation_of_oxygen_in_sea_water
found 1132 dataset(s)
	 C02 SBE16 Oxygen http://www.neracoos.org/erddap/tabledap/C02_sbe16_disox_all
	 URI GSO Pier, Surface Exo http://www.neracoos.org/erddap/tabledap/URI_GSOPier_ExoTop
	 URI 168 Mount View, Bottom Sonde http://www.neracoos.org/erddap/tabledap/URI_168-MV_BottomSonde
	 Great Bay,NH. Squamscott River WQ station http://www.neracoos.org/erddap/tabledap/GRBSQWQ_NERRS
	 URI Conimicut Point  Bottom Sonde http://www.neracoos.org/erddap/tabledap/URI_CPoint_BottomSonde
	 Great Bay,NH. Great Bay WQ station http://www.neracoos.org/erddap/tabledap/GRBGBWQ_NERRS
	 A01 SBE16 Oxygen http://www.neracoos.org/erddap/tabledap/A01_sbe16_disox_all
	 URI Poppasquash Point Bottom Sonde http://www.neracoos.org/erddap/tabledap/URI_PP_BottomSonde
	 URI Greenwich Bay, Surface Sonde http://www.neracoos.org/erddap/ta